In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/My\ Drive/books

/content/drive/My Drive/books


In [3]:
!pip install tashaphyne
!pip install pyarabic

In [4]:
!ls

alldata		  dictionary.json  epochBIG.h5	Naftawayh-0.1  textdata
arabic_const.py   encoder	   finalData	preText.py
dictionary2.json  epochBIG	   finalData2	__pycache__


In [0]:
import os


In [0]:
# d='textdata/{}'
# for file in os.listdir("textdata/JOR"):
#     if file.endswith(".txt"):
#         print(os.path.join("textdata", file))

In [0]:
def read_file(D):
  f=open(D,'r',encoding='utf-8').read()
  return f.split(' ')

In [8]:
from preText import Text_clean,saveFile
finaldatadir='finalData2/'
dic='textdata/'
savedir='alldata/'
for files in os.listdir(dic):
  for txtfile in os.listdir(dic + files):
      if txtfile.endswith(".txt"):
         D = os.path.join(dic + '/' + files , txtfile)
         l = saveFile.read_file(D)
         #print(l[:10])
         words = Text_clean.CleanThewordsList(l)
         saveFile.to_text(words,'{}{}.txt'.format(finaldatadir,str(files)))
         #print('saved = ' , str(D))

             

  

IndexError: ignored

In [0]:

# data_dict={}
# for f in os.listdir('finalData/'):
#   _l=saveFile.to_list('finalData/{}'.format(f))
#   data_dict[f.split('.')[0]]=_l

# all_data=[]
# for i in data_dict.keys():
#   all_data = all_data + data_dict[i]
# print(len(all_data))
# uniqeset=list(set(all_data))
# print(len(uniqeset))

In [0]:
# from collections import Counter 
  
# def most_frequent(List): 
#     occurence_count = Counter(List) 
#     return occurence_count.most_common(20)
# #most_frequent(all_data)

In [0]:
x_=[]
y_=[]
for f in os.listdir(finaldatadir):
  _l=saveFile.to_list('{}{}'.format(finaldatadir,f))
  for i in range(10,len(_l),10):
    x_.append(_l[i-10:i])
    y_.append(f.split('.')[0])


In [0]:
len(x_)

In [0]:
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer

In [0]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(x_)

In [0]:
# dictionary = tokenizer.word_index
# # Let's save this out so we can use it later
# with open('dictionary2.json', 'w') as dictionary_file:
#     json.dump(dictionary, dictionary_file)

In [0]:
sequences = tokenizer.texts_to_sequences(x_)

In [0]:
len(sequences)

In [0]:
vocabulary_size = len(tokenizer.word_counts)
vocabulary_size

In [0]:
import numpy as np

In [0]:
X = np.array(sequences)

In [0]:
 from sklearn.preprocessing import LabelBinarizer
 encoder = LabelBinarizer()
 Y = encoder.fit_transform(y_)

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding,Dropout

In [0]:
def create_model(vocabulary_size, seq_len,output_size):
    model = Sequential()
    model.add(Embedding(vocabulary_size, 10, input_length=seq_len))
    model.add(LSTM(150, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(150))
    model.add(Dropout(0.3))

    model.add(Dense(150, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(40, activation='relu'))
    model.add(Dense(output_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
             X, Y, test_size=0.10,shuffle=True)

In [0]:
model=create_model(vocabulary_size+1,10,5)

In [0]:
hist=model.fit(X_train, y_train, batch_size=30, epochs=10,verbose=1,validation_data=[X_test,y_test])

In [0]:
import matplotlib.pyplot as plt

plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


In [0]:

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.show()


In [0]:
from pickle import dump,load

In [0]:
# save the model to file
model.save('model2.h5')
# save the tokenizer
dump(tokenizer, open('model2tok', 'wb'))
dump(encoder, open('model2enc', 'wb'))

In [0]:
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [0]:
# encoded_text = tokenizer.texts_to_sequences([input_text])[0]
# pad_encoded = pad_sequences([encoded_text], maxlen=10, truncating='pre')
# pre=model.predict(pad_encoded)
# print(pad_encoded)
# print(encoder.inverse_transform(pre))

In [0]:
allpre=model.predict(X_test)

In [0]:
preclass=encoder.inverse_transform(allpre)
y_true=encoder.inverse_transform(y_test)

In [0]:
from sklearn.metrics import classification_report,confusion_matrix

In [0]:
print(classification_report(y_true,preclass))

In [0]:
import pandas as pd
import seaborn as sns
arr=confusion_matrix(y_true,preclass)
print(arr)
row,col = np.diag_indices(arr.shape[0])
arr[row,col]=200
df = pd.DataFrame(arr, index = [i for i in ['AR','EG','JOR','MOR','SA']],
                  columns = [i for i in ['AR','EG','JOR','MOR','SA']])
plt.figure(figsize = (15,13))
sns.heatmap(df, annot=True)

In [0]:
# fallpre=[]
# for c,i in enumerate(X_test):
#   pad=pad_sequences([i], maxlen=10, truncating='pre')
#   p=model.predict(pad)
#   pe=encoder.inverse_transform(p)
#   y_true = encoder.inverse_transform(y_test[c:c+1])
#   if pe !=y_true:
#     fallpre.append(i)

In [0]:
# fallpre[0]

In [0]:
# len(fallpre)

In [0]:
# for i in fallpre[80]:
#   print(tokenizer.index_word[i] )